<a href="https://colab.research.google.com/github/harinijs03/2023103549_SDC_assignment/blob/main/Reatilerstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
from io import BytesIO
import base64

# --- Sample Customer Data (REPLACE WITH YOUR ACTUAL DATA) ---
data = {
    'CustomerID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'Avg_Spend': [50, 80, 60, 120, 40, 90, 70, 55, 110, 65, 95, 75, 100, 45, 85],
    'Frequency': [5, 10, 7, 15, 3, 12, 8, 6, 14, 9, 11, 8, 13, 4, 10],
    'Recency_Days': [30, 10, 20, 5, 45, 8, 15, 25, 7, 18, 12, 22, 9, 35, 16],
    'Satisfaction': [4, 5, 4, 5, 3, 5, 4, 4, 5, 4, 5, 4, 5, 3, 4]
}
df = pd.DataFrame(data).set_index('CustomerID')

# --- Preprocessing: Scale Numerical Features ---
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

def analyze_customer_groups(n_clusters_slider):
    try:
        n_clusters = int(n_clusters_slider)
        if n_clusters <= 0:
            return "Number of clusters must be greater than 0.", None, None

        # Perform Agglomerative Clustering
        agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
        clusters = agg_clustering.fit_predict(scaled_df)
        clustered_df = df.copy()
        clustered_df['Cluster'] = clusters

        # Analyze cluster characteristics (mean values)
        cluster_analysis = clustered_df.groupby('Cluster').mean().to_html(float_format="%.2f")

        # Generate Dendrogram (for visualization of the hierarchy)
        linked = linkage(scaled_df, 'ward')
        plt.figure(figsize=(10, 7))
        dendrogram(linked, orientation='top', labels=df.index.tolist())
        plt.title('Hierarchical Clustering Dendrogram')
        plt.xlabel('Customer ID')
        plt.ylabel('Distance')
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image_base64 = base64.b64encode(buf.read()).decode('utf-8')
        plt.close()
        image = f'<img src="data:image/png;base64,{image_base64}" alt="Dendrogram">'

        interpretation = "Based on the cluster means, you can understand the characteristics of different customer segments."

        return interpretation, image, cluster_analysis

    except ValueError:
        return "Please enter a valid number for the number of clusters.", None, None
    except Exception as e:
        return f"An error occurred: {e}", None, None

iface = gr.Interface(
    fn=analyze_customer_groups,
    inputs=gr.Slider(2, 5, 1, label="Number of Clusters"),  # Changed to positional arguments
    outputs=[
        gr.Textbox(label="Cluster Interpretation"),
        gr.HTML(label="Dendrogram"),
        gr.HTML(label="Cluster Analysis (Mean Values)")
    ],
    title="Retail Customer Grouping (Hierarchical Clustering)",
    description="Group retail customers based on their spending habits, frequency, recency, and satisfaction using Hierarchical Clustering."
)
iface.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.8 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d65fddda4bcb9eec6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
